In [1]:
import sys
import os
import cobra
import libsbml
from cobra import Model, Reaction, Metabolite, manipulation, test
import pandas as pd
import csv
import copy
from pprint import pprint
from tabulate import tabulate



#Change working dir first, ty ChatGPT, much loves
cwd = os.getcwd()
# Split the path into a list of directories
directories = cwd.split(os.sep)
# Remove the last two directories from the list
directories = directories[:-2]
# Join the directories back into a path
new_cwd = os.sep.join(directories)
# Change the current working directory to the new path
os.chdir(new_cwd)

sys.path.append("./src")

import model_manipulation  as mm



In [2]:
ios2164_orig = cobra.io.read_sbml_model('./model/ios2164_orig.xml')

ios2164_orig
inf = 1e6

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-03


In [3]:
#Check mass balance of original ios2164 model

imbalanced_rxns = list()

non_exchange_rxns = set(ios2164_orig.reactions) - set(ios2164_orig.boundary) - set(ios2164_orig.demands)

for rxns in non_exchange_rxns:
    #Check if rxns are not in boundary
        #Retrieve mass balance dictionary
        mb_dict = rxns.check_mass_balance()
        #Check if imbalanced + if it has H+
        if len(mb_dict) != 0:
            imbalanced_rxns.append(rxns)
        

#149 imbalanced rxns that are in boundary
#636 reactions that are within the cell. Let's check which of these are imbaalanced.
print('initial imbalanced reactions length for ios2164_orig:', len(imbalanced_rxns))

initial imbalanced reactions length for ios2164_orig: 647


In [82]:
def rebalance_model(model):
# #Balance reactions from the inactivated reactions file to see if it fixes yung internal cycling issues
    # #rebalance rubisco
    # ios2164_orig.metabolites.cdpdag_os_r.formula = 'C14H18N3O15P2R2'
    # ios2164_orig.metabolites.dhqrctn_c.formula = 'C15H11O7'
    # ios2164_orig.metabolites.oopc4coa_x.formula = 'C35H52N7O19P3S'
    # ios2164_orig.metabolites.oopc4coa_x.formula = 'C35H52N7O19P3S'
    # ios2164_orig.metabolites.udpkrmn_c.formula = 'C15H22N2O16P2'
    # ios2164_orig.metabolites.udprmn_c.formula = 'C15H22N2O16P2'
    ios2164_orig.metabolites.cgly_c.formula = 'C5H10N2O3S' #Not encoded properly
    ios2164_orig.metabolites.cholp_r.formula = 'C5H13NO4P'
    ios2164_orig.metabolites.cholp_s.formula = 'C5H13NO4P'
    ios2164_orig.metabolites.dtmp_s.formula = 'C10H13N2O8P' #Was not encoded properly
    ios2164_orig.metabolites.get_by_id('5dpmev_c').formula = 'C6H10O10P2'
    ios2164_orig.metabolites.lnlncoa_r.formula = 'C39H62N7O17P3S'
    ios2164_orig.metabolites.lnlnlACP_s.formula = 'C29H52N2O8PRS'
    ios2164_orig.metabolites.lnlnl_c.formula = 'C18H29O2'
    ios2164_orig.metabolites.lnlnl_s.formula = 'C18H29O2'
    ios2164_orig.metabolites.lnlnl_x.formula = 'C18H29O2'
    ios2164_orig.metabolites.o2_u.formula = 'O2' #Not encoded properly
    ios2164_orig.metabolites.plast_s.formula = 'C53H82O2'
    
    
    ios2164_orig.reactions.AASADc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.AATCm.add_metabolites({'h_m':-1})
    ios2164_orig.reactions.AATCs.add_metabolites({'h_s':-1})
    ios2164_orig.reactions.ACDOc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.ACGAMKc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.ACHBSs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.ACHYSLc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.ACRSPc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.ACRSPc.add_metabolites({'h_c':-1})  
    ios2164_orig.reactions.ACSs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.ADEDAc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.ADMATs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.ADPARTs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.ALATm.add_metabolites({'h_m':1})
    ios2164_orig.reactions.ALATs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.ALTNDAc.add_metabolites({'h_c':-2.0})
    ios2164_orig.reactions.AMETAHYCSTc.add_metabolites({'h_c':1,  'h2o_c':1})
    ios2164_orig.reactions.ANTPPTs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.ANXANDEOs.add_metabolites({'h_s':2})
    ios2164_orig.reactions.AOBUTDx.add_metabolites({'h_x':1})
    ios2164_orig.reactions.APPRTs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.ARGSSs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.ARGTs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.ARGTs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.ASCOXc.add_metabolites({'h_c':8})
    ios2164_orig.reactions.ASCPO1c.add_metabolites({'h_c':4})
    ios2164_orig.reactions.ASCPO1s.add_metabolites({'h_s':2})
    ios2164_orig.reactions.GTHDHc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.ASCPO2c.add_metabolites({'h_c':1})
    ios2164_orig.reactions.ASCPO2s.add_metabolites({'h_s':3})
    ios2164_orig.reactions.ASNS1s.add_metabolites({'h_s':1})
    ios2164_orig.reactions.ASNTLs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.ASNtm.add_metabolites({'h_m':1})
    ios2164_orig.reactions.ASPALs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.ASPCLc.add_metabolites({'h_c': -1})
    ios2164_orig.reactions.ASPCT1c.add_metabolites({'h_c':1})
    ios2164_orig.reactions.ASPCTs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.ASPHGNAHr.add_metabolites({'h_r':-1})
    ios2164_orig.reactions.ASPHGNAr.add_metabolites({'h_r':-1})
    ios2164_orig.reactions.ASPHGNAHr.add_metabolites({'h_r':-1}) 
    ios2164_orig.reactions.ASPTLm.add_metabolites({'h_m':1})
    ios2164_orig.reactions.ASPTLs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.ATAHc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.ATPAPc.reaction = 'atp_c + 2.0 h2o_c --> amp_c + 2.0 pi_c + 2.0 h_c'
    ios2164_orig.reactions.ATPSs.add_metabolites({'h_u':-1})
    ios2164_orig.reactions.BRS2c.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.C3STDH2c.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.C4STMO1c.add_metabolites({'h_c':1})
    ios2164_orig.reactions.CAFFMTc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.CAVNS2s.add_metabolites({'h_s':2})
    ios2164_orig.reactions.CBMKs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.CBPSs.add_metabolites({'h_s':3})
    ios2164_orig.reactions.CC3H1c.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.CDPDAGSm.add_metabolites({'h_m':1})
    ios2164_orig.reactions.CELLSc.add_metabolites({'h_c': 2})
    ios2164_orig.reactions.CHALIc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.CHOLPCTs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.CKDH1c.add_metabolites({'h_c':1})
    ios2164_orig.reactions.CKDH2c.add_metabolites({'h_c':1})
    ios2164_orig.reactions.CKDH3c.add_metabolites({'h_c':1})
    
    ios2164_orig.reactions.CKGT1c.add_metabolites({'h_c':1})
    ios2164_orig.reactions.CKGT2c.add_metabolites({'h_c':1})
    ios2164_orig.reactions.CKGT3c.add_metabolites({'h_c':1})
    ios2164_orig.reactions.CKGT4c.add_metabolites({'h_c':1})
    ios2164_orig.reactions.CKGT5c.add_metabolites({'h_c':1})
    ios2164_orig.reactions.CKGT6c.add_metabolites({'h_c':1})
    ios2164_orig.reactions.CKGT7c.add_metabolites({'h_c':1})
    ios2164_orig.reactions.CKGT8c.add_metabolites({'h_c':1})
    ios2164_orig.reactions.CKOGT1c.add_metabolites({'h_c':1})
    ios2164_orig.reactions.CKOGT2c.add_metabolites({'h_c':1})
    ios2164_orig.reactions.CKOGT3c.add_metabolites({'h_c':1})
    ios2164_orig.reactions.CKPc.add_metabolites({'h_c':2})
    ios2164_orig.reactions.CMPAc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.CNDNG3Gc.add_metabolites({'h_c': 2})
    ios2164_orig.reactions.CNDNG5Gc.add_metabolites({'h_c':2})
    ios2164_orig.reactions.CNDNGLT5c.add_metabolites({'h_c':2})
    ios2164_orig.reactions.COUMGUTFc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.COUMNSc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.CTPCHc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.CYSSs.add_metabolites({'h_s':-1})
    ios2164_orig.reactions.CYSTLm.add_metabolites({'h_m':-1})
    ios2164_orig.reactions.CYTTHs.bounds = (-inf, 0)
    ios2164_orig.reactions.DAGLc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.DCTPDAc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.DGSNTTPc.add_metabolites({'h_c':-2})
    ios2164_orig.reactions.DHDHc.add_metabolites({'h_c':2})
    ios2164_orig.reactions.DHDPSs.add_metabolites({'h_s': -4})
    ios2164_orig.reactions.DHKFORc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.DHKRc.add_metabolites({'h_c': 1})
    ios2164_orig.reactions.DHNCOAHs.add_metabolites({'h_s': 1})
    ios2164_orig.reactions.DHNCOASs.add_metabolites({'h_s': -1})
    ios2164_orig.reactions.DHNOPTs.add_metabolites({'h_s': 2})
    ios2164_orig.reactions.DHNPPTs.add_metabolites({'h_s':2})
    ios2164_orig.reactions.DHORSts.add_metabolites({'h_s':-1})
    ios2164_orig.reactions.DHPDc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.DHPRs.add_metabolites({'h_s': 2})
    ios2164_orig.reactions.DHPRys.add_metabolites({'h_s': 2})
    ios2164_orig.reactions.DHPSm.add_metabolites({'h_m':1})
    ios2164_orig.reactions.DHQORc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.DHQRc.add_metabolites({'h2o_c':-1})
    ios2164_orig.reactions.DMATTs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.DMPPTs.add_metabolites({'h_s': 1})
    ios2164_orig.reactions.DMPQMTs.add_metabolites({'h_s': 1})
    ios2164_orig.reactions.DNTPPAc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.EAR160ym.add_metabolites({'h_m':-1})
    ios2164_orig.reactions.EAR160ys.add_metabolites({'h_s':-1})
    ios2164_orig.reactions.ECASDATEO1c.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.ECASDATEO3c.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.ECASDO1c.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.ECASDO3c.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.ECASDSs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.EIKAUROc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.EIKAURSs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.EKAURSs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.ENTBSs.delete #Remove bacterial reaction w/ no GPR
    ios2164_orig.reactions.ESANDO1c.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.ESANDSs.add_metabolites({'h_s':-1})
    ios2164_orig.reactions.ESANDSs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.ETHAPTr.add_metabolites({'h_r':1})
    ios2164_orig.reactions.FA140COAabcx.add_metabolites({'h2o_x':-1,'h_x':1})
    ios2164_orig.reactions.FA160ACOASc.add_metabolites({'h_s':-1})
    ios2164_orig.reactions.FA160ACPHs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.FA160COAabcx.add_metabolites({'h2o_x':-1,'h_x':1})
    ios2164_orig.reactions.FA161COAabcx.add_metabolites({'h2o_x':-1,'h_x':1})
    ios2164_orig.reactions.FA180COAabcx.add_metabolites({'h2o_x':-1,'h_x':1})
    ios2164_orig.reactions.FA181COAabcx.add_metabolites({'h2o_x':-1,'h_x':1})
    ios2164_orig.reactions.FA182ACOASc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.FA182ACPHs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.FA182COAabcx.add_metabolites({'h2o_x':-1,'h_x':1})
    ios2164_orig.reactions.FA183ACOASc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.FA183ACPHs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.FA183COAabcx.add_metabolites({'h2o_x':-1,'h_x':1})
    ios2164_orig.reactions.FA200COAHr.add_metabolites({'h_r':1})
    ios2164_orig.reactions.FA220COAHr.add_metabolites({'h_r':1})
    ios2164_orig.reactions.FA240COAHr.add_metabolites({'h_r':1})
    ios2164_orig.reactions.FA260COAHr.add_metabolites({'h_r':1})
    ios2164_orig.reactions.FA280COAHr.add_metabolites({'h_r':1})
    ios2164_orig.reactions.FA300COAHr.add_metabolites({'h_r':1})
    ios2164_orig.reactions.FAS161COA_Lc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.FAS181ACP_Lm.add_metabolites({'h_m':-1})
    ios2164_orig.reactions.FAS181ACP_Ls.add_metabolites({'h_s':-1})
    ios2164_orig.reactions.FAS181COA_Lc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.FAS182ACP_Ls.add_metabolites({'h_s':-2})
    ios2164_orig.reactions.FAS182COA_Lc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.FAS183ACP_Ls.add_metabolites({'h_s':1})
    ios2164_orig.reactions.FAS183COA_Lc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.FERHc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.FGFTm.add_metabolites({'h_m':1})
    ios2164_orig.reactions.FGFTs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.FMNATc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.FMNATm.add_metabolites({'h_m':2})
    ios2164_orig.reactions.FMNATs.add_metabolites({'h_s':2})
    ios2164_orig.reactions.FNORs.add_metabolites({'h_s':2})
    ios2164_orig.reactions.FORMHLc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.FRDPFTs.add_metabolites({'h_s': 5})
    ios2164_orig.reactions.FRPTc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.FRTTs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.FTHFCLc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.FTHFCLm.add_metabolites({'h_m':1})
    ios2164_orig.reactions.FTHFCLs.add_metabolites({'h_c':1})
    ios2164_orig.reactions.GA4DOc.add_metabolites({'h2o_c':1, 'h_c':1})
    ios2164_orig.reactions.GAAKORG10c.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.GAAKORG11c.add_metabolites({'h2o_c':1})
    ios2164_orig.reactions.GAAKORG12c.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.GAAKORG8c.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.GAAKORG9c.add_metabolites({'h2o_c':1})
    ios2164_orig.reactions.GAMPTc.add_metabolites({'h_c':1.0})
    ios2164_orig.reactions.GBDPs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.GCOADCx.add_metabolites({'h_x':-1})
    ios2164_orig.reactions.GDPMDHc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.GENT2c.add_metabolites({'h_c':1})
    ios2164_orig.reactions.GENT3c.add_metabolites({'h_c':1})
    ios2164_orig.reactions.GLCNSc.add_metabolites({'h_c':2})
    ios2164_orig.reactions.GLUTRSs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.GLYATx.add_metabolites({'h_x':-1})
    ios2164_orig.reactions.GLYB1s.add_metabolites({'h_s':2})
    ios2164_orig.reactions.GLYDHDm.add_metabolites({'h_m':-1})
    ios2164_orig.reactions.GLYTLm.add_metabolites({'h_m':1})
    ios2164_orig.reactions.GLYTLs.add_metabolites({'h_m':1})
    ios2164_orig.reactions.GMPc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.GNSKc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.GRTTm.add_metabolites({'h_m':1})
    ios2164_orig.reactions.GRTTs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.GSNPc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.GSNRHc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.GTHDHc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.GTHDHs.add_metabolites({'h_s':-1})
    ios2164_orig.reactions.GTPCYTDPTc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.GTPCYTDPTs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.GTPDHs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.GTPDPKs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.H2Sts.add_metabolites({'h_s':1})
    ios2164_orig.reactions.HCYSMT2c.add_metabolites({'h_c':1})
    ios2164_orig.reactions.HDHs.add_metabolites({'h_s': -1})
    ios2164_orig.reactions.HEMEOSm.add_metabolites({'h_m':1})
    ios2164_orig.reactions.HFERMTc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.HISTLm.add_metabolites({'h_m':1})
    ios2164_orig.reactions.HISTLs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.HMGCOARc.add_metabolites({'h_c': -1.0})
    ios2164_orig.reactions.HMGCOARyc.add_metabolites({'h_c': -1.0})
    ios2164_orig.reactions.HMGLm.add_metabolites({'h_m':1})
    ios2164_orig.reactions.HOXGs.add_metabolites({'h_s':-2})
    ios2164_orig.reactions.HPPKm.add_metabolites({'h_m':1})
    ios2164_orig.reactions.IAAAMYHc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.IAAIASc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.IAAIGSc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.ILETLm.add_metabolites({'h_m':1})
    ios2164_orig.reactions.ILETLs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.IPDPT1s.add_metabolites({'h_s':2})
    ios2164_orig.reactions.IPDPT2s.add_metabolites({'h_s':1})
    ios2164_orig.reactions.IPKK2c.add_metabolites({'h_c':-2})
    ios2164_orig.reactions.IPKK4c.add_metabolites({'h_c':-2})
    ios2164_orig.reactions.IPKK5c.add_metabolites({'h_c':-2})
    ios2164_orig.reactions.IPKK6c.add_metabolites({'h_c': -2.0})
    ios2164_orig.reactions.ITK1Kc.add_metabolites({'h_c':-2})
    ios2164_orig.reactions.LCYCORc.add_metabolites({'h_c': 1})
    ios2164_orig.reactions.LCYSTSm.add_metabolites({'h_m':2})
    ios2164_orig.reactions.LCYSTSs.add_metabolites({'h_s':2})
    ios2164_orig.reactions.LIMSs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.LNL9HPLs.add_metabolites({'h_s':-2})
    ios2164_orig.reactions.LPLGDOc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.LYSTLm.add_metabolites({'h_m':1})
    ios2164_orig.reactions.LYSTLs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.MACPCLm.add_metabolites({'h_m':-2})
    ios2164_orig.reactions.MACPCLs.add_metabolites({'h_s':-1})
    ios2164_orig.reactions.MACPMMEc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.MAGLc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.MANKINc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.MARMSc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.MCOATAc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.MCOATAm.add_metabolites({'h_m':1})
    ios2164_orig.reactions.MCOATAs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.MDHARm.add_metabolites({'h_m':-2})
    ios2164_orig.reactions.MDHARyc.add_metabolites({'h_c':-2})
    ios2164_orig.reactions.MDHARym.add_metabolites({'h_m':-2})
    ios2164_orig.reactions.MECDPDH5s.add_metabolites({'h_s':2})
    ios2164_orig.reactions.MEPCTs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.METSc.add_metabolites({'h_c':1.0})
    ios2164_orig.reactions.METSs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.METTLm.add_metabolites({'h_m':1})
    ios2164_orig.reactions.METTLs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.MGCHLs.add_metabolites({'h_s': 2})
    ios2164_orig.reactions.MGCm.add_metabolites({'h_m':1})
    ios2164_orig.reactions.MI1345PP2c.add_metabolites({'h_c':-2})
    ios2164_orig.reactions.MI1345PPc.add_metabolites({'h_c': -2.0})
    ios2164_orig.reactions.MI1346PK1c.add_metabolites({'h_c':-2})
    ios2164_orig.reactions.MI1346PK2c.add_metabolites({'h_c':-2})
    ios2164_orig.reactions.MI1456PKc.add_metabolites({'h_c':-2})
    ios2164_orig.reactions.MI34Pc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.MLTHFR_Fc.add_metabolites({'h_c':3})
    ios2164_orig.reactions.MLTHFRc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.MLTHFRs.add_metabolites({'h_s':-1})
    ios2164_orig.reactions.MOMASs.add_metabolites({'h_s':-1})
    ios2164_orig.reactions.MOMASys.add_metabolites({'h_s':-1})
    ios2164_orig.reactions.MPMECY1s.add_metabolites({'h_s':-1})
    ios2164_orig.reactions.MPMECY2s.add_metabolites({'h_s':2})
    ios2164_orig.reactions.MPMTs.add_metabolites({'h_s': -1})
    ios2164_orig.reactions.MRs.add_metabolites({'h_s':2})
    ios2164_orig.reactions.MSBQMTs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.MTAMnh4m.add_metabolites({'h_m':1})
    ios2164_orig.reactions.NADOR2u.add_metabolites({'h_u':-1.0})
    ios2164_orig.reactions.NNAMc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.NNDPRs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.NO3Rm.add_metabolites({'h_m':-2})
    ios2164_orig.reactions.NOSc.reaction = '2 arg-L_c + 3 nadph_c + 4 o2_c --> 2 citr-L_c + 3 nadp_c + 2 no_c + 4 h2o_c'
    ios2164_orig.reactions.NRGNAORc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.NRGNMTc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.NRGNNORc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.OCTDPOTs.add_metabolites({'h_s': 1})
    ios2164_orig.reactions.ORPRTs.add_metabolites({'h_s':-1})
    ios2164_orig.reactions.OXAACc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.P450MO1c.add_metabolites({'h_c':- 1})
    ios2164_orig.reactions.P450MO2c.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.P450MO3c.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.P450MO4c.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.P450MO5c.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.P450MO6c.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.PCAHr.add_metabolites({'h_r':-1})
    ios2164_orig.reactions.PEDCm.add_metabolites({'h_m':1})
    ios2164_orig.reactions.PEDCr.add_metabolites({'h_r':1})
    ios2164_orig.reactions.PEPSs.add_metabolites({'h_s':2})
    ios2164_orig.reactions.PHENH4Lc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.PHETLm.add_metabolites({'h_m':1})
    ios2164_orig.reactions.PHETLs.add_metabolites({'h_s':1}) 
    ios2164_orig.reactions.PHYTS1s.add_metabolites({'h_s':1})
    ios2164_orig.reactions.PHYTS2s.add_metabolites({'h_s':1})
    ios2164_orig.reactions.PINCLc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.PLIBTNSm.add_metabolites({'h_m':-3})
    ios2164_orig.reactions.PLIVLPNSm.add_metabolites({'h_m':-3})
    ios2164_orig.reactions.PPASm.add_metabolites({'h_m':-1})
    ios2164_orig.reactions.PPASs.add_metabolites({'h_s':-1})
    ios2164_orig.reactions.PPDKc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.PPDKs.add_metabolites({'h_s':2})
    ios2164_orig.reactions.PPPGPPPc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.PRADPs.add_metabolites({'h_s': 1})
    ios2164_orig.reactions.PRAICs.add_metabolites({'h_s':-1})
    ios2164_orig.reactions.PRDPARs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.PRFGSs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.PRIMADNSs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.PRIMARTSc.add_metabolites({'h_s':2})
    ios2164_orig.reactions.PROTLm.add_metabolites({'h_m':1})
    ios2164_orig.reactions.PROTLs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.QRCTNGLT3c.add_metabolites({'h_c': 1})
    ios2164_orig.reactions.RBPCs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.RIBFKc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.RIBFSs.add_metabolites({'h_s':2})
    ios2164_orig.reactions.SELNPS.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.SERTLm.add_metabolites({'h_m':1})
    ios2164_orig.reactions.SERTLs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.SHCHFs.add_metabolites({'h_s':3})
    ios2164_orig.reactions.SOXs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.SQTs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.STEMOc.add_metabolites({'h_c': -1})
    ios2164_orig.reactions.STEMSs.add_metabolites({'h_s': 1})
    ios2164_orig.reactions.SUATs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.SULRs.add_metabolites({'h_s':6})
    ios2164_orig.reactions.TAGLc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.THR3DHx.add_metabolites({'h_x':-1.0})
    ios2164_orig.reactions.THRTLm.add_metabolites({'h_m':1})
    ios2164_orig.reactions.TMDPKc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.TMPPPs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.TRICINS1c.add_metabolites({'h_c':1})
    ios2164_orig.reactions.TRICINS2c.add_metabolites({'h_c':1})
    ios2164_orig.reactions.TRPTLm.add_metabolites({'h_m':1})
    ios2164_orig.reactions.TRPTLs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.TYRLs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.TYRNH4Lc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.TYRTLm.add_metabolites({'h_m':1})
    ios2164_orig.reactions.TYRTLs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.UAMASc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.UDPGALMc.add_metabolites({'h_c':-2})
    ios2164_orig.reactions.UDPGALMc.add_metabolites({'h_c':-2})
    ios2164_orig.reactions.UDPGALts.add_metabolites({'h_s':1})
    ios2164_orig.reactions.UDPGATc.add_metabolites({'h_c':1})
    ios2164_orig.reactions.UDPGts.add_metabolites({'h_s':1})
    ios2164_orig.reactions.UDPSQSs.add_metabolites({'h_s':-1})
    ios2164_orig.reactions.UPRTs.add_metabolites({'h_s':-1})
    ios2164_orig.reactions.URDGLAHc.add_metabolites({'h_c':-1})     
    ios2164_orig.reactions.UREASEc.add_metabolites({'h_c':-2.0})
    ios2164_orig.reactions.UROc.add_metabolites({'h_c':-1})
    ios2164_orig.reactions.VALTLm.add_metabolites({'h_m':1})
    ios2164_orig.reactions.VALTLs.add_metabolites({'h_s':1})
    ios2164_orig.reactions.VLXANDEOs.add_metabolites({'h_s':2})
    ios2164_orig.reactions.XSc.add_metabolites({'h_c':2})
    ios2164_orig.reactions.get_by_id('2HEDBT8GTc').add_metabolites({'h_c':1})
    ios2164_orig.reactions.get_by_id('2HNDBT8GTc').add_metabolites({'h_c':1})
    ios2164_orig.reactions.get_by_id('2HNDGDHc').add_metabolites({'h_c':1})
    ios2164_orig.reactions.get_by_id('2SBZCOALs').add_metabolites({'h_s': 1})
    ios2164_orig.reactions.get_by_id('3OAS180m').add_metabolites({'h_m':1})
    ios2164_orig.reactions.get_by_id('3OAS180s').add_metabolites({'h_s':1})
    ios2164_orig.reactions.get_by_id('SUCL(ADP)m').add_metabolites({'h_m':-1})
    ios2164_orig.reactions.get_by_id('SUCL(GDP)m').add_metabolites({'h_m':-1})

#     ios2164_orig.metabolites.cdpdag_os_r.formula = 'C14H18N3O15P2R2'
#     ios2164_orig.metabolites.ocdcya_s.formula = 'C18H29O2' #Doesn't work

In [83]:
imbalanced_rxns = list()
print(len(imbalanced_rxns))


non_exchange_rxns = set(ios2164_orig.reactions) - set(ios2164_orig.boundary) - set(ios2164_orig.demands)


#Check mass balance of original ios2164 model
with ios2164_orig as ios2164_orig_copy:
    rebalance_model(ios2164_orig_copy)
#     ios2164_orig_copy.reactions.DHDHc.add_metabolites({'h_c':2})
    for rxns in non_exchange_rxns:
            #Retrieve mass balance dictionary
            mb_dict = rxns.check_mass_balance()
            #Check if imbalanced 
            if len(mb_dict) != 0:
                imbalanced_rxns.append(rxns)
                

#149 imbalanced rxns that are in boundary
#636 reactions that are within the cell. Let's check which of these are imbaalanced.
print('imbalanced reactions length for ios2164_orig after rebalancing:', len(imbalanced_rxns))

0
imbalanced reactions length for ios2164_orig after rebalancing: 324


In [84]:
for items in imbalanced_rxns:
    print(items.id,items.check_mass_balance())

PSIINC {'X': -2.0}
SELCYSTLc {'charge': 1.0}
AGPATs {'charge': -1.0, 'H': -1.0, 'O': 1.0, 'R': 1.0}
IORNTNGTc {'charge': -1.0}
SELCYSTLs {'charge': 1.0}
VTXNSc {'charge': -1.0}
ACHMSSCs {'charge': -1.0}
ACSERLc {'charge': 2.0, 'H': 1.0}
FRNCTIc {'H': 2.0}
DOLKc {'charge': -1.0}
DEDOLDPSATc {'charge': 2.0}
TRICINGTc {'charge': -1.0}
2HNDBT8GTc {'charge': 1.0}
GLCNACPTc {'H': -107.0, 'C': -63.0, 'N': -1.0, 'O': -12.0, 'P': -2.0, 'X': 1.0}
AGPATm {'charge': -1.0, 'H': -1.0, 'O': 1.0, 'R': 1.0}
UPP1Rs {'charge': 8.0}
FAS182ACP_Ls {'charge': 2.0, 'H': 4.0}
CNDNG3Gc {}
NOSc {'H': -3.0}
FAS183ACP_Ls {'charge': 1.0, 'H': 1.0}
ACCOAC_OSAr {'charge': 3.9996, 'C': -16.4075, 'H': -30.5912, 'N': 0.000700000000000145, 'O': -0.9982999999999969, 'P': 0.0002999999999997449, 'S': 9.999999999998899e-05}
ACOATAs {'charge': 1.0}
ACPC_OSm {'charge': 0.3809, 'C': -16.4085, 'H': -31.94290000000001, 'N': 0.00019999999999997797, 'O': -0.9992000000000001, 'P': 9.999999999998899e-05, 'R': 9.999999999998899e-05, '

In [85]:
ios2164_orig.reactions.MM1c

Reaction identifier,MM1c
Name,"mannosyl-oligosaccharide 1,2-alpha-mannosidase (G00012)"
Memory address,0x07fc018ca9ee0
Stoichiometry,G00011_c + 4.0 h2o_c --> G00012_c + 4.0 man_c (GlcNAc)2 (Man)9 (Asn)1 + 4.0 H2O --> (GlcNAc)2 (Man)5 (Asn)1 + 4.0 D-Mannose
GPR,LOC_Os04g51690 or LOC_Os05g11850
Lower bound,0.0
Upper bound,1000000000.0


In [87]:
ios2164_orig.metabolites.G00011_c

Metabolite identifier,G00011_c
Name,(GlcNAc)2 (Man)9 (Asn)1
Memory address,0x07fc019d6faf0
Formula,X
Compartment,c
In 3 reaction(s),"MG3Bc, MM1c, MG3Ac"


In [ ]:
ios2164_2cell = cobra.io.read_sbml_model('./model/ios2164_2cell.xml')
ios2164_2cell.reactions.DM_Phloem_BS

#I need to modify the objective function since Starch is being exported from M cell to BS cell. 

In [ ]:
at_core_model = cobra.io.read_sbml_model('./Supplementary_PrevLit/Blaetke & Brautigam/data/2018-23-05-mb-genC3.sbml')
internal_rxns = set(at_core_model.reactions) - set(at_core_model.boundary)

unbalanced_rxns = list()
for rxns in internal_rxns:
    if len(rxns.check_mass_balance()) != 0:
#         print(rxns.id, '\t', rxns.check_mass_balance())
        unbalanced_rxns.append(rxns)
    
    
at_core_model
    

In [ ]:
print('internal reactions in B&B core AT model: ', len(internal_rxns))
print('unbalanced rxns in B&B: ', len(unbalanced_rxns))
print('percent unbalanced: ', len(unbalanced_rxns)/len(internal_rxns) * 100)

#IN the case of B&B's model puro charge imbalance yung nagaganap. Yung sa case ko kasi karamihan is mass imbalance talaga.

In [ ]:
#How about yung core Arabidopsis model nina Shameer et al (2019) -- yung C3 mitochondria
shameer_model = cobra.io.read_sbml_model('./Supplementary_PrevLit/Shameer (2019)/core_model.xml')
internal_rxns = set(shameer_model.reactions) - set(shameer_model.boundary)


unbalanced_rxns = list()
for rxns in internal_rxns:
    if len(rxns.check_mass_balance()) != 0:
#         print(rxns.id, '\t', rxns.check_mass_balance())
        unbalanced_rxns.append(rxns)
    
print('internal reactions in Shameer core AT model: ', len(internal_rxns))
print('unbalanced rxns in Shameer AT model: ', len(unbalanced_rxns))
print('percent unbalanced: ', len(unbalanced_rxns)/len(internal_rxns) * 100)

#IN the case of B&B's model puro charge imbalance yung nagaganap. Yung sa case ko kasi karamihan is mass imbalance talaga.

In [ ]:
for items in unbalanced_rxns:
    print(items.id, items.check_mass_balance())

In [ ]:
shameer_model.reactions.Plastidial_ATP_Synthase_p

In [ ]:
shameer_model.reactions.RXN_7678_NADP_p

Notes:
Weirdly enough the Shameer model is even much more imbalanced compared dun sa current model ko, which I believe is more of an encoding issue as opposed with actual mass-charge violations, considering that there weren't much changes done by the author in their supplementary scripts. 
Meanwhile, the core Arabidopsis model used by B&B doesn't have much mass-charge violations owing to the small size of the model.
So far after implementing constraints on cofactor recycling (based on the Boolean logic algorithm formulated by Lakshmanan in the original iOS2164) there seems to be limited evidence of any type II or type III internal loops with pFBA. Furthermore, there is concordance with the scale of flux values when comparing with the experimentation by Shameer et al in both the purely linear diel model and the hybrid kinetic setup (2018 and 2019), indicating that the model produces relatively realistic ATP rates and other things. 